In [ ]:
import tkinter as tk
from tkinter import filedialog, scrolledtext, messagebox
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# ---------- Linecounter Class ----------
class Linecounter:
    def __init__(self, file_name):
        self.file_name = file_name
        self.line = []

    def read(self):
        with open(self.file_name, "r") as f:
            self.line = f.readlines()

    def fetch_id_add(self):
        return [x.split(" ")[0] for x in self.line]

    def ip_less_than(self, threshold):
        result = []
        for line in self.line:
            ip = line.split(" ")[0]
            if "." in ip:
                parts = ip.split(".")
                if parts[0].isdigit() and int(parts[0]) < threshold:
                    result.append(ip)
            elif ip.isdigit() and int(ip) < threshold:
                result.append(ip)
        return result

    def find_ratio(self, threshold=20):
        total = len(self.line)
        lt_count = len(self.ip_less_than(threshold))
        return lt_count / total if total else 0

# ---------- Tkinter App ----------
root = tk.Tk()
root.title("IP Analyzer")
root.geometry("950x900")

output = scrolledtext.ScrolledText(root, width=100, height=15)
output.pack(pady=10)

status_label = tk.Label(root, text="", fg="blue")
status_label.pack()

canvas_frame = tk.Frame(root)
canvas_frame.pack(pady=10)

# ---------- Functions ----------
def clear_canvas():
    for widget in canvas_frame.winfo_children():
        widget.destroy()

def select_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        global counter
        counter = Linecounter(file_path)
        counter.read()
        status_label.config(text="File loaded successfully.", fg="green")

def show_all_ips():
    if not hasattr(counter, "line"):
        status_label.config(text="Load a file first.", fg="red")
        return
    ips = counter.fetch_id_add()
    output.delete(1.0, "end")
    output.insert("end", "\n".join(ips))

def show_ips_lt_n():
    if not hasattr(counter, "line"):
        status_label.config(text="Load a file or use manual input first.", fg="red")
        return
    try:
        n = int(lt_entry.get())
    except ValueError:
        status_label.config(text="Enter a valid integer.", fg="red")
        return

    result = counter.ip_less_than(n)
    output.delete(1.0, "end")
    output.insert("end", "\n".join(result))
    status_label.config(text=f"{len(result)} IPs found with first octet < {n}", fg="blue")

def show_summary():
    if not hasattr(counter, "line"):
        status_label.config(text="Load a file first.", fg="red")
        return
    total = len(counter.line)
    all_ips = counter.fetch_id_add()
    try:
        threshold = int(lt_entry.get())
    except:
        threshold = 20  # Default
    lt = counter.ip_less_than(threshold)
    ratio = counter.find_ratio(threshold)
    summary = (
        f"Total Lines: {total}\n"
        f"Total IPs: {len(all_ips)}\n"
        f"IPs < {threshold}: {len(lt)}\n"
        f"Ratio: {ratio:.2f}"
    )
    output.delete(1.0, "end")
    output.insert("end", summary)

def export_results():
    data = output.get("1.0", "end").strip()
    if not data:
        status_label.config(text="No data to export.", fg="red")
        return
    path = filedialog.asksaveasfilename(defaultextension=".txt")
    if path:
        with open(path, "w") as f:
            f.write(data)
        status_label.config(text=f"Results saved to {path}", fg="green")

def plot_ratio():
    if not hasattr(counter, "line"):
        status_label.config(text="Load a file first.", fg="red")
        return
    try:
        threshold = int(lt_entry.get())
    except:
        threshold = 20
    clear_canvas()
    lt_count = len(counter.ip_less_than(threshold))
    total = len(counter.line)
    other = total - lt_count

    fig = Figure(figsize=(4, 3), dpi=100)
    ax = fig.add_subplot(111)
    ax.pie([lt_count, other], labels=[f"< {threshold}", "Other"], autopct="%1.1f%%")
    ax.set_title(f"IP < {threshold} Ratio")

    canvas = FigureCanvasTkAgg(fig, master=canvas_frame)
    canvas.draw()
    canvas.get_tk_widget().pack()

def search_ip():
    term = search_entry.get()
    output.tag_remove("highlight", "1.0", "end")
    if not term:
        return
    pos = "1.0"
    while True:
        pos = output.search(term, pos, stopindex="end")
        if not pos:
            break
        end = f"{pos}+{len(term)}c"
        output.tag_add("highlight", pos, end)
        pos = end
    output.tag_config("highlight", background="yellow", foreground="black")

def analyze_manual():
    text = manual_input.get("1.0", "end").strip().splitlines()
    if not text:
        status_label.config(text="Manual input is empty.", fg="red")
        return
    global counter
    counter = Linecounter("manual.txt")
    counter.line = text
    try:
        threshold = int(lt_entry.get())
    except:
        threshold = 20
    ratio = counter.find_ratio(threshold)
    output.delete(1.0, "end")
    output.insert("end", f"Manual Input IP < {threshold} Ratio: {ratio:.2f}")
    status_label.config(text="Manual input processed.", fg="green")

def filter_range():
    if not hasattr(counter, "line"):
        status_label.config(text="Load a file or use manual input first.", fg="red")
        return
    try:
        min_val = int(min_entry.get())
        max_val = int(max_entry.get())
    except ValueError:
        status_label.config(text="Enter valid integer bounds.", fg="red")
        return

    if min_val > max_val:
        status_label.config(text="Min should be less than Max.", fg="red")
        return

    filtered_ips = []
    for x in counter.line:
        ip = x.split(" ")[0]
        if "." in ip:
            first = ip.split(".")[0]
            if first.isdigit() and min_val <= int(first) <= max_val:
                filtered_ips.append(ip)
        elif ip.isdigit() and min_val <= int(ip) <= max_val:
            filtered_ips.append(ip)

    output.delete(1.0, "end")
    output.insert("end", "\n".join(filtered_ips))
    status_label.config(text=f"Found {len(filtered_ips)} IPs in range.", fg="blue")

# ---------- Buttons ----------
btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

tk.Button(btn_frame, text="Select File", command=select_file).grid(row=0, column=0, padx=5)
tk.Button(btn_frame, text="Show All IPs", command=show_all_ips).grid(row=0, column=1, padx=5)
tk.Button(btn_frame, text="Summary", command=show_summary).grid(row=0, column=2, padx=5)
tk.Button(btn_frame, text="Export", command=export_results).grid(row=0, column=3, padx=5)
tk.Button(btn_frame, text="Plot", command=plot_ratio).grid(row=0, column=4, padx=5)

# ---------- Dynamic < N Filter ----------
threshold_frame = tk.Frame(root)
threshold_frame.pack(pady=5)
tk.Label(threshold_frame, text="Show IPs with first octet < ").grid(row=0, column=0)
lt_entry = tk.Entry(threshold_frame, width=10)
lt_entry.insert(0, "20")
lt_entry.grid(row=0, column=1)
tk.Button(threshold_frame, text="Filter", command=show_ips_lt_n).grid(row=0, column=2, padx=5)

# ---------- Search ----------
search_entry = tk.Entry(root)
search_entry.pack(pady=5)
search_entry.insert(0, "Search IP")
tk.Button(root, text="Highlight", command=search_ip).pack(pady=2)

# ---------- Manual Input ----------
tk.Label(root, text="Manual Input:").pack()
manual_input = scrolledtext.ScrolledText(root, width=100, height=5)
manual_input.pack(pady=5)
tk.Button(root, text="Analyze Manual Input", command=analyze_manual).pack(pady=5)

# ---------- Range Filter ----------
tk.Label(root, text="Filter IPs in Range:").pack(pady=5)
range_frame = tk.Frame(root)
range_frame.pack()

tk.Label(range_frame, text="Min:").grid(row=0, column=0)
min_entry = tk.Entry(range_frame, width=10)
min_entry.grid(row=0, column=1)

tk.Label(range_frame, text="Max:").grid(row=0, column=2)
max_entry = tk.Entry(range_frame, width=10)
max_entry.grid(row=0, column=3)

tk.Button(range_frame, text="Filter Range", command=filter_range).grid(row=0, column=4, padx=10)

# ---------- Start the GUI ----------
root.mainloop()
